# Imports

In [1]:
from modules.data import data_loader, data_splitter, data_reader
from modules.models import arima, lag_llama
from modules.evaluation import evaluate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from modules.experiment.experiment import run_experiment
from modules.data.tscv import create_tscv_dataset
from modules.experiment.tscv import get_tscv_results, mean_directional_accuracy


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from itertools import islice
from sklearn.model_selection import TimeSeriesSplit

# Parameters

In [2]:
PREDICTION_LENGTH = 10 # currentlz only works for PREDICTION_LENGTH > 1
FREQUENCY = "daily" # currently we only have dailz frequency
TYPE_OF_DATA = "stock" # currently we only have stock prices saved
MODELS = ["arima", "llama"] # currentlz works onlz for these two

# want to add
PREDICTION_HORIZON = 3 # can use anything as long as it complies with data length
#TRAIN_PERIOD = # context lenghts. Should take a look into this
#TRAIN_SIZE = 
FOLDS = 10# for TSCV
CONTEXT_LENGTH = 64



In [2]:
series1 = pd.Series([1, 2, 3])
series2 = pd.Series([5,6, 3])

mean_directional_accuracy(series1, series2)

0.5

# Notes

 - autoregressor
 - mean directional accuracy
 - ask
 - for each time-series create a whole dataframe for TSCV
  - problem with this is with time horizon that is >1 
  - prediction horizon is only for that value in the future

# TO DO

 - create the TSCV experiment. It needs to create a table. Metrics for each fold
 - lit review
 - content for the presentation by monday!!
 - review data leakage risk
 - writing of the dissertation
 - autoregressor
 - MDA metric
 - frequency

 Writing
 - lit review
 - lit reading

# Loading the data

In [3]:
# loading the data for all 500 S&P500 stocks
data = data_reader.read_data(TYPE_OF_DATA)[0]
simple_data = data_loader.get_simle_data()

[*********************100%%**********************]  1 of 1 completed


# Prepare the data

In [4]:
# pass full data at maximum granularity and produce according to frequency parameter

In [6]:
tscv_data = create_tscv_dataset(data = simple_data, context_length=CONTEXT_LENGTH, n_folds=FOLDS, prediction_horizon=PREDICTION_HORIZON, max_folds=False)

# Models

In [7]:
# arima


In [8]:
# lag llama

# Run regular experiment

In [ ]:
results = run_experiment(data = data, prediction_length = PREDICTION_LENGTH)

# Evaluation

In [ ]:
results

# Run TSCV experiment

look up tzpes of TSCV

In [10]:
tscv_results, prediction = run_experiment(data=tscv_data, tscv=True)

y0 done  1
------------
y1 done  2
------------
y2 done  3
------------
y3 done  4
------------
y4 done  5
------------
y5 done  6
------------
y6 done  7
------------
y7 done  8
------------
y8 done  9
------------
y9 done  10
------------


In [11]:
tscv_results

{'r2': {'arima': -0.8036159605688358, 'llama': -4.690321882253741},
 'mse': {'arima': 9.014638013311012, 'llama': 28.440750730306206},
 'mae': {'arima': 2.712121007999187, 'llama': 4.178632465124013},
 'rmse': {'arima': 3.0024386776936867, 'llama': 5.332987036390226}}

# SKLearn TSCV

In [3]:
simple_data = data_loader.get_simle_data()

[*********************100%%**********************]  1 of 1 completed


In [4]:
# sklearn tscv object
tscv = TimeSeriesSplit(n_splits=FOLDS, test_size=PREDICTION_LENGTH)


In [5]:
# extracting the time-series data from original data
series = simple_data["y"]

- put metrics in dataframe. one for arima, one for lag llama
- each column metric, each row is a fold
- create wrapper function get_metrics(predicted, actual) returns pd.DataFrame of description above
- by end of day have a table
- agenda of the presentation by 12
- implement autoregressor forecast t1 = t0
- implement frequency experimentation
- start training date global parameter

In [6]:
# initializing the list of models, metrics and emptz result dict
models=["arima", "llama"]
metrics=["r2", "mse", "mae", "rmse", "mda"]
results = {metric: {model: {f"fold_{i}": [] for i in range(FOLDS)} for model in models} for metric in metrics}

arima_results = pd.DataFrame(columns=metrics)

llama_results = pd.DataFrame(columns= metrics)




i = 0
# iterating over all the folds
for train_index, test_index in tscv.split(series):
    # subsetting the original data according to train/test split
    train = simple_data.iloc[train_index]
    valid = list(simple_data.iloc[test_index]["y"])

    # for my testing purposes
    #print(train["y"])
    #print(train["y"].iloc[-1])


    # inputting data into the models
    arima_model = arima.get_autoarima(train)
    autoarima_predictions = arima.autoarima_predictions(arima_model, PREDICTION_LENGTH)
    lag_llama_predictions, tss = lag_llama.get_lam_llama_forecast(train, PREDICTION_LENGTH, context_length=CONTEXT_LENGTH)
    lag_llama_predictions = list(lag_llama_predictions[0].samples.mean(axis = 0))

    """
    print(i)
    print("----------")
    print(len(lag_llama_predictions))
    print("--------------")
    print(len(valid))
    print("---------------")
    print(PREDICTION_LENGTH)
    print("------------------")
    print(r2_score(valid, lag_llama_predictions))
    """

    # for my own testing purposes
    """
    print(autoarima_predictions)
    print(lag_llama_predictions)
    print(valid)
    """

    # recording the metrics
    """
    results["r2"]["arima"][f"fold_{i}"].append(r2_score(valid, autoarima_predictions))
    results["mse"]["arima"][f"fold_{i}"].append(mean_squared_error(valid, autoarima_predictions))
    results["mae"]["arima"][f"fold_{i}"].append(mean_absolute_error(valid, autoarima_predictions))
    results["rmse"]["arima"][f"fold_{i}"].append(np.sqrt(mean_squared_error(valid, autoarima_predictions)))

    results["r2"]["llama"][f"fold_{i}"].append(r2_score(valid, lag_llama_predictions))
    results["mse"]["llama"][f"fold_{i}"].append(mean_squared_error(valid, lag_llama_predictions))
    results["mae"]["llama"][f"fold_{i}"].append(mean_absolute_error(valid, lag_llama_predictions))
    results["rmse"]["llama"][f"fold_{i}"].append(np.sqrt(mean_squared_error(valid, lag_llama_predictions)))
    """

    arima_metrics = [r2_score(valid, autoarima_predictions), 
               mean_squared_error(valid, autoarima_predictions), 
               mean_absolute_error(valid, autoarima_predictions),
               np.sqrt(mean_squared_error(valid, autoarima_predictions)),
               mean_directional_accuracy(valid, autoarima_predictions, train["y"].iloc[-1])]
    
    llama_metrics = [r2_score(valid, lag_llama_predictions), 
               mean_squared_error(valid, lag_llama_predictions), 
               mean_absolute_error(valid, lag_llama_predictions),
               np.sqrt(mean_squared_error(valid, lag_llama_predictions)),
               mean_directional_accuracy(valid, lag_llama_predictions, train["y"].iloc[-1])]

    arima_results = pd.concat([arima_results, pd.DataFrame([arima_metrics], columns=metrics)], ignore_index=True)
    llama_results = pd.concat([llama_results, pd.DataFrame([llama_metrics], columns=metrics)], ignore_index=True)


    i += 1

0
----------
10
--------------
10
---------------
10
------------------
-9.88214173756848


C:\Users\topco\AppData\Local\Temp\ipykernel_18956\3132376272.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  arima_results = pd.concat([arima_results, pd.DataFrame([arima_metrics], columns=metrics)], ignore_index=True)
C:\Users\topco\AppData\Local\Temp\ipykernel_18956\3132376272.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  llama_results = pd.concat([llama_results, pd.DataFrame([llama_metrics], columns=metrics)], ignore_index=True)


1
----------
10
--------------
10
---------------
10
------------------
-7.331208574748089
2
----------
10
--------------
10
---------------
10
------------------
-23.111337830449575
3
----------
10
--------------
10
---------------
10
------------------
-105.08256584719334
4
----------
10
--------------
10
---------------
10
------------------
-14.882898517829759
5
----------
10
--------------
10
---------------
10
------------------
-1.7788387313339746
6
----------
10
--------------
10
---------------
10
------------------
-5.033386216002038
7
----------
10
--------------
10
---------------
10
------------------
-13.167825665726498
8
----------
10
--------------
10
---------------
10
------------------
-7.01598773131259
9
----------
10
--------------
10
---------------
10
------------------
-28.119125267629563


In [7]:
arima_results

,r2,mse,mae,rmse,mda
0,-10.146053,33.415565,5.512960,5.780620,0.2
1,-1.069529,56.237796,5.979982,7.499186,0.8
2,-0.021127,5.389042,1.626925,2.321431,0.4
3,-0.130501,2.120663,1.296481,1.456249,0.6
4,0.009028,14.090569,3.312309,3.753741,0.5
5,-1.962910,94.272351,7.710001,9.709395,0.4
6,-2.799649,39.613602,5.353981,6.293934,0.1
7,-0.137856,7.045263,2.276720,2.654291,0.9
8,-1.762008,200.092281,12.148269,14.145398,0.4
9,-0.884450,68.486433,6.201827,8.275653,0.1


In [8]:
llama_results

,r2,mse,mae,rmse,mda
0,-9.882142,32.624367,5.155265,5.711774,0.5
1,-7.331209,226.393918,13.871125,15.046392,0.6
2,-23.111338,127.248596,10.969855,11.280452,0.6
3,-105.082566,198.996161,13.894630,14.106600,0.4
4,-14.882899,225.838039,14.093629,15.027909,0.6
5,-1.778839,88.415652,8.128944,9.402960,0.6
6,-5.033386,62.901640,7.252293,7.931055,0.5
7,-13.167826,87.722892,8.714343,9.366050,0.3
8,-7.015988,580.714283,22.745289,24.098014,0.6
9,-28.119125,1058.274159,31.907340,32.531126,0.6


In [9]:
np.mean(llama_results["mda"])

0.5299999999999999

In [4]:
r = get_tscv_results(simple_data, PREDICTION_HORIZON, CONTEXT_LENGTH, FOLDS)

C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(5,0,0)(0,0,0)[0]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
                           ^^^^^^^^^^^^^^
  File "C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
             ^^^^^^^^^^
  File "C:\Users\topco\AppDat

In [5]:
r

{'r2': {'arima': {'fold_0': [-3.691877661141132],
   'fold_1': [-0.010542006383208902],
   'fold_2': [-5.611156232296624],
   'fold_3': [-2.0666207482946946],
   'fold_4': [-0.7687106529191812],
   'fold_5': [-0.3431518091946324],
   'fold_6': [-5.351096735700188],
   'fold_7': [-1.6938904126202843],
   'fold_8': [-0.011681380345721193],
   'fold_9': [-4.330864243718463]},
  'llama': {'fold_0': [-9.336629997575274],
   'fold_1': [-179.7260621892066],
   'fold_2': [-107.60443413307539],
   'fold_3': [-115.009555818314],
   'fold_4': [-4.876441039021601],
   'fold_5': [-304.6272485868018],
   'fold_6': [-67.11577380450797],
   'fold_7': [-120.96634096008222],
   'fold_8': [-102.0076103887577],
   'fold_9': [-178.49412755448967]}},
 'mse': {'arima': {'fold_0': [13.875785862988721],
   'fold_1': [0.31214081152770684],
   'fold_2': [5.642126724971973],
   'fold_3': [2.991871755262922],
   'fold_4': [123.78820905652573],
   'fold_5': [3.945837239182269],
   'fold_6': [51.01199731770291],
   